# RSA Implementation and common mistakes

## How to set-up RSA for Dummies
The first problem we have is generating primes of sufficient size. Let's start by making a little function to check if big numbers are prime

In [13]:
from random import randint
smallPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 
            127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 
            257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 
            401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

def isPrime(n):
    # Quickly test small factors.
    if n in smallPrimes:
            return True

    # Use Fermat's little theorem to do the harder work on larger primes. (https://en.wikipedia.org/wiki/Fermat%27s_little_theorem)
    for i in range(0, 50): 
        base = randint(2, n-1)
        if pow(base, n-1, n) != 1: 
            return False
    
    return True

And let's check we've not been completely stupid by checking a few cases:

In [2]:
print(2, isPrime(2))
print(3, isPrime(3))
print(10, isPrime(10))

print(65537, isPrime(65537))
print(65539, isPrime(65539))
print(65541, isPrime(65541))



2 True
3 True
10 False
65537 True
65539 True
65541 False


And now for something a bit bigger. 

This prime was nicked from the [RFC 3526](https://www.rfc-editor.org/rfc/rfc3526.txt). 

In [3]:
bigPrime = """FFFFFFFF FFFFFFFF C90FDAA2 2168C234 C4C6628B 80DC1CD1
      29024E08 8A67CC74 020BBEA6 3B139B22 514A0879 8E3404DD
      EF9519B3 CD3A431B 302B0A6D F25F1437 4FE1356D 6D51C245
      E485B576 625E7EC6 F44C42E9 A637ED6B 0BFF5CB6 F406B7ED
      EE386BFB 5A899FA5 AE9F2411 7C4B1FE6 49286651 ECE45B3D
      C2007CB8 A163BF05 98DA4836 1C55D39A 69163FA8 FD24CF5F
      83655D23 DCA3AD96 1C62F356 208552BB 9ED52907 7096966D
      670C354E 4ABC9804 F1746C08 CA18217C 32905E46 2E36CE3B
      E39E772C 180E8603 9B2783A2 EC07A28F B5C55DF0 6F4C52C9
      DE2BCBF6 95581718 3995497C EA956AE5 15D22618 98FA0510
      15728E5A 8AACAA68 FFFFFFFF FFFFFFFF"""

bigPrime = bigPrime.replace(" ", "")
bigPrime = bigPrime.replace("\n", "")

bigDaddy = int(bigPrime, 16)

isPrime(bigDaddy)

True

Alright, so how do we generate a large random prime. The code below implements the process:

In [10]:
def bigPrime(bits):
    from random import getrandbits
    p = getrandbits(bits)
    while not isPrime(p):
        p = getrandbits(bits)
    return p

Does this prove that the numbers are prime? Of course not, they're sort of "industrial strength" primes.

Alrighty then, let's generate some primes:

In [5]:
p = bigPrime(1024)
q = bigPrime(1024)

p,q

(51402346371786911924859657627877264174318641051044733116035120148279170128536109769281435040734816214937867077513413198789270012902272337871859111430009340709709229044577573001052653645222168901691268112706295236194871941197010119846240088444006712177103461957976000784684915575356592878960100191991971011731,
 70392417262343667923260905711576696724552943748866557066696783979420889679515640637463040136932257543510808912988714008376993095009746439712228561982675169700863911791457864449236150669975379255169655022063651499610609921391139830122142703303465399418302814136335620488946000385714097643057012743544427069739)

And multiply to get a modulus:

In [6]:
n = p * q
n

3618335414066341426393494370108640853068300663472767604615808119735503157334339693929725415792829564142944905879628043058284505011721265149747369245148625205843094386162783190831059205291140679420675911164936767963521064572219838890694607630972266404388807658077187034479571819812969748320474906296476473030866660410997533882555186088697356153394732831523180206640073058904114007280170425987553084867042862645477823854381685416391167119233925330904135116579564142569472627940460659936731778245280274474582470479680539419706729329680112979071052708128726773496625767260458095129452891443762604412309579464739524108209

# Explanation of the role of Extended GCD / LCM

The extended Euclian algorithm is the standard GCD algorithm extended to show more of its internal state. It's used to find solutions to the equations of the form:

$$(ax + by) = gcd(x, y)$$

In our case, we're looking for a solution such that:

$$e \times d \equiv {1 \mod {\text{lcm} (p-1,q-1)}}$$

So, if we substitute:

$$ex + (b \times {\text{lcm} (p-1,q-1)}) = gcd(e, {\text{lcm} (p-1,q-1)})$$

But by requirement for the scheme to work:

$$gcd(e, {\text{lcm} (p-1,q-1)}) = 1$$

So we can rewrite the right hand side to 1:

$$(e \times x) + (b \times {\text{lcm} (p-1,q-1)}) = 1$$

Now we take both sides  $\mod {\text{lcm} (p-1,q-1)}$:

$$((e \times x) + (b \times {\text{lcm} (p-1,q-1)})) \mod {\text{lcm} (p-1,q-1)} \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

But notice that we just have $b$ multiplied by the thing in the mod, so we can simplify:

$$ex \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

This looks like our target equation above, so $x=d$ which is our encryption exponent.

In [15]:
def gcdExtended(a, b):
		if a == 0 :   
			return b,0,1
		     
		gcd,x1,y1 = gcdExtended(b % a, a)  
     
		x = y1 - (b//a) * x1  
		y = x1  
	     
		return gcd,x,y 

def lcm(a, b):
	gcd, _, _ = gcdExtended(a, b)
	return (a * b) // gcd


Right, now we get to the business end of setting up RSA. First we need to compute phi(n), which is the number of numbers relatively prime to "n"

In [8]:
phi_n = lcm(p-1, q-1)

Next we need to define what we want for the encryption exponent. It needs to be big enough so that when we do m^e it is bigger than p and thus "loops around"

In [9]:
e = 65537

Now the final step, let's compute d from phi_n and e.

In [10]:
_, d, _ = gcdExtended(e, phi_n)

if d < 0:
	d = phi_n + d # d is negative. Lost a good 45 minutes there.

d

580622005615586304082688916844645338225624974097248279810521477809457542367007696281672297102175749749138350896182436102086592260185340875342450961542419045382744130980681285783219160664117688559554728165251041706034290180718524431908537790120387561862076319272605359538953828265912329801673472268594760648608089774943882552343239003882179202700500859074496201533845434704154252995873110635909085612681698640927140905297608586101190213697099810305480551994514391131712871448703600291113074070154492749444110237770301477947532981181204955911290235677914226343655870406774484524844771734496994254043681039021692705803

Alright, let's check everything works!!!

In [11]:
m = 420

cipherText = pow(m, e, n)
plainText = pow(cipherText, d, n)

m, e, plainText

(420, 65537, 420)

# Now let's put the whole thing in to one neat package

In [4]:
class RSAFixedKey:
    def __init__(self, e=65537):
        self.p = bigPrime(1024)
        self.q = bigPrime(1024)
        self.n = self.p * self.q
        self.e = e
        phi_n = lcm((self.p-1),(self.q-1))
        _, self.d, _ = gcdExtended(e, phi_n)
        if self.d < 0:
            self.d = phi_n + self.d

    def encrypt(self, data):
        return pow(data, self.e, self.n)

    def decrypt(self, data):
        return pow(data, self.d, self.n)

And let's test it out!

In [13]:
rsa = RSAFixedKey()

plainTextBefore = 69
cipherText=rsa.encrypt(69)
plainTextAfter=rsa.decrypt(cipherText)

plainTextBefore, cipherText, plainTextAfter


(69,
 10381887356601132798020334627436706624842029712095337646696557740980093156011625747075751062186937254403309397368128943163258998894335715432471884013127971725077602037365314966589211967031092537233102648028338243571417623711113691488694006765883976427971441057233409799470429184602949814121077286382553982729493366415544490829762174521101373809199778469806690726134423774411719269464598788669799985826331712268078559945600713534705455171063114551917306051756076550057956438612941394611149008218938464409224587537402613195393273912547007769291259081660920857822286009060480253477742642741004628134815281151003824724342,
 69)

# Attack: Not picking a big enough exponent for $e$

In [14]:
rsa = RSAFixedKey(e=3)

In [15]:
from SemanticSecurityGame import SemanticSecurityGame

class RSAGameStrategyLowExponent:
    def getMessages(self, trialNumber):
           m0 = 3
           m1 = 2
           
           return m0, m1

    def challenge(self, challenge, trialNumber):
           result = challenge ** (1/3)

           if result == 3:
                return 0 
           else:
                return 1

strategy = RSAGameStrategyLowExponent()
game = SemanticSecurityGame(rsa, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 50}

# Attack: The cipher-text is the same each time!

In [17]:
rsa = RSAFixedKey()

In [17]:
from SemanticSecurityGame import SemanticSecurityGame

class RSAGameStrategy:
    def getMessages(self, trialNumber):
            if trialNumber == 0:
                m0 = 69
                m1 = 69
            else:
                m0 = 69
                m1 = 420
            return m0, m1 

    def challenge(self, challenge, trialNumber):
            if (trialNumber == 0):
                self.savedMessage = challenge
            if self.savedMessage == challenge:
                return 0
            else:
                return 1

strategy = RSAGameStrategy()
            
game = SemanticSecurityGame(rsa, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 49}

# Attack: Algebraic properties of RSA

In [18]:
m_0 = 69
m_1 = 420

m_2 = 420*69
m_2

28980

And now we find the cipher-text for $c_2$ from the cipher-texts from $m_0$ and $m_1$.

In [20]:
c_0 = rsa.encrypt(m_0)
c_1 = rsa.encrypt(m_1)

c_2 = c_0*c_1 % rsa.n

rsa.decrypt(c_2)

28980

# How to use RSA properly

https://en.wikipedia.org/wiki/Optimal_asymmetric_encryption_padding